In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.cluster import KMeans
from scipy import stats
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import silhouette_score, silhouette_samples
import matplotlib.cm as cm
from sklearn.impute import KNNImputer

# Read the data
df = pd.read_csv('/work/DM2425_ABCDEats_DATASET (1).csv')

# Strip trailing spaces from column names
df.columns = df.columns.str.strip()

# Check for duplicates
df.duplicated().sum()

# Drop duplicates
df.drop_duplicates(inplace=True)

# Function to fix HR_0 values
def fix_hr0(df):
    """
    Fix HR_0 values by comparing total DOW orders with sum of HR orders.
    If there's a mismatch, assign missing orders to HR_0.
    """
    # Calculate total orders from DOW columns
    dow_cols = ['DOW_0', 'DOW_1', 'DOW_2', 'DOW_3', 'DOW_4', 'DOW_5', 'DOW_6']
    total_dow_orders = df[dow_cols].sum(axis=1)
    
    # Calculate total orders from HR columns (excluding HR_0)
    hr_cols = [f'HR_{i}' for i in range(1, 24)]  # HR_1 to HR_23
    total_hr_orders = df[hr_cols].sum(axis=1)
    
    # Calculate missing orders
    missing_orders = total_dow_orders - total_hr_orders
    
    # Update HR_0 with missing orders where the difference is positive
    df['HR_0'] = np.where(missing_orders > 0, missing_orders, 0)
    
    # Print summary of changes
    total_fixed = (missing_orders > 0).sum()
    total_orders_added = missing_orders[missing_orders > 0].sum()
    
    print(f"Fixed {total_fixed} rows")
    print(f"Added {total_orders_added} orders to HR_0")
    print("\nVerification:")
    print(f"Total DOW orders: {total_dow_orders.sum()}")
    print(f"Total HR orders after fix: {df[['HR_' + str(i) for i in range(24)]].sum().sum()}")
    
    return df

# Apply the HR_0 fix
df = fix_hr0(df)

# Convert data types
df['customer_age'] = pd.to_numeric(df['customer_age'], errors='coerce').astype('Int64')
df['first_order'] = pd.to_numeric(df['first_order'], errors='coerce').astype('Int64')
df['HR_0'] = pd.to_numeric(df['HR_0'], errors='coerce').astype('Int64')

# Check for null values
df.isna().sum()

# KNN imputation function
def knn_imputation(df, columns=['customer_age', 'first_order'], n_neighbors=5):
    """
    Perform KNN imputation with NA handling
    """
    # Convert NA/None to np.nan
    for col in columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Create KNN imputer
    imputer = KNNImputer(n_neighbors=n_neighbors)
    
    # Perform imputation
    imputed_data = imputer.fit_transform(df[columns])
    
    # Update DataFrame
    for i, column in enumerate(columns):
        df[column] = imputed_data[:, i]
    
    return df

# Apply KNN imputation
df = knn_imputation(df)

# Replace missing values in last_promo
df['last_promo'].replace('-', 'None')

# Convert datatypes after imputation
df['customer_age'] = df['customer_age'].astype('int64')
df['first_order'] = df['first_order'].astype('int64')

# Convert CUI columns to int
cui_columns = [col for col in df.columns if col.startswith('CUI_')]
df[cui_columns] = df[cui_columns].astype('int64')

# Remove customers with 0 vendors
print(f"Number of rows before removing 0-vendor customers: {len(df)}")
df = df[df['vendor_count'] > 0]
print(f"Number of rows after removing 0-vendor customers: {len(df)}")

# Reset index
df = df.reset_index(drop=True)

df_original = pd.read_csv('/work/DM2425_ABCDEats_DATASET (1).csv')

/tmp/ipykernel_1147/728900981.py:14: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/work/DM2425_ABCDEats_DATASET (1).csv')
Fixed 1164 rows
Added 1552 orders to HR_0

Verification:
Total DOW orders: 139263
Total HR orders after fix: 139263
Number of rows before removing 0-vendor customers: 31875
Number of rows after removing 0-vendor customers: 31737
/tmp/ipykernel_1147/728900981.py:111: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv('/work/DM2425_ABCDEats_DATASET (1).csv')


In [2]:
# Remove customers with 0 vendors
print(f"Number of rows after removing 0-vendor customers: {len(df)}")
df = df[df['vendor_count'] > 0]

# Verify the removal
print(f"Number of rows after removing 0-vendor customers: {len(df)}")

# Optional: Reset the index if needed
df = df.reset_index(drop=True)

Number of rows after removing 0-vendor customers: 31737
Number of rows after removing 0-vendor customers: 31737


In [3]:
 #Customer Value Features
# Total monetary value
cuisine_cols = [col for col in df.columns if col.startswith('CUI_')]
df['total_spending'] = df[cuisine_cols].sum(axis=1)
df['total_orders'] = df[[col for col in df.columns if col.startswith('DOW_')]].sum(axis=1)

# Number of different cuisines ordered (variety)
df['cuisine_diversity'] = (df[cuisine_cols] > 0).sum(axis=1)

# Calculate total orders
df['total_orders'] = df[[col for col in df.columns if col.startswith('DOW_')]].sum(axis=1)

# Get primary cuisine (highest spending category)
df['primary_cuisine'] = df[cuisine_cols].idxmax(axis=1).str.replace('CUI_', '')

   # Weekend vs Weekday preference
df['weekend_orders'] = df['DOW_0'] + df['DOW_6']
df['underweekend_orders'] = df['DOW_1'] + df['DOW_2'] + df['DOW_3'] + df['DOW_4'] + df['DOW_5']
df['weekend_ratio'] = df['weekend_orders'] / df['total_orders'].replace(0, 1)


# Peak hours preference (lunch 11-14, dinner 18-21)
lunch_hours = ['HR_11', 'HR_12', 'HR_13']
dinner_hours = ['HR_18', 'HR_19', 'HR_20']

df['lunch_orders'] = df[lunch_hours].sum(axis=1)
df['dinner_orders'] = df[dinner_hours].sum(axis=1)

# Primary meal time preference
df['meal_preference'] = np.where(df['lunch_orders'] > df['dinner_orders'], 'lunch', 'dinner')
    


# Are they ordering throughout the day or only specific times?
morning_hours = ['HR_6', 'HR_7', 'HR_8', 'HR_9', 'HR_10']
midday_hours = ['HR_11', 'HR_12', 'HR_13', 'HR_14', 'HR_15']
evening_hours = ['HR_16', 'HR_17', 'HR_18', 'HR_19', 'HR_20']
night_hours = ['HR_21', 'HR_22', 'HR_23', 'HR_0']
drinking_hours = [ 'HR_1','HR_2', 'HR_3', 'HR_4', 'HR_5']

df['morning_orders'] = df[morning_hours].sum(axis=1)
df['midday_orders'] = df[midday_hours].sum(axis=1)
df['evening_orders'] = df[evening_hours].sum(axis=1)
df['night_orders'] = df[night_hours].sum(axis=1)
df['drinking_orders'] = df[drinking_hours].sum(axis=1)

# Calculate primary time period
time_periods = ['morning_orders', 'midday_orders', 'evening_orders', 'night_orders', 'drinking_orders']
df['primary_time_period'] = df[time_periods].idxmax(axis=1).str.replace('_orders', '')



#New Features: CUI_NewAsian, CUI_NewOther, 
#df['CUI_NewAsian']= df['CUI_Japanese'] + df['CUI_Chinese'] + df['CUI_Indian'] + df['CUI_Thai']
#df['CUI_NewOther'] = df['CUI_Desserts'] + df['CUI_Cafe'] + df['CUI_Street Food / Snacks'] + df['CUI_Beverages']
#Left: American, Chicken dishes, Healthy, Noodle Dishes, Italian 
          

In [4]:
# Save the processed DataFrame to a CSV file
df.to_csv('processed_ABCDEats_data.csv', index=False)

# Alternatively, you can save it as a pickle file which preserves data types better
df.to_pickle('processed_ABCDEats_data.pkl')

# Save the processed DataFrame to a CSV file
df_original.to_csv('original_ABCDEats_data.csv', index=False)

# Alternatively, you can save it as a pickle file which preserves data types better
df_original.to_pickle('original_ABCDEats_data.pkl')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d80cf8a3-0707-4865-ba36-9382bf28b032' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>